<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
%reload_ext tensorboard

### Load data from csv

In [3]:
X = pd.read_csv('data.csv')
X.head()

,f1,f2,label
0,0.450564,1.074305,0.0
1,0.085632,0.967682,0.0
2,0.117326,0.971521,1.0
3,0.982179,-0.380408,0.0
4,-0.720352,0.955850,0.0


In [4]:
Y = X['label']
print(Y)

0        0.0
1        0.0
2        1.0
3        0.0
4        0.0
        ... 
19995    0.0
19996    1.0
19997    1.0
19998    0.0
19999    0.0
Name: label, Length: 20000, dtype: float64


In [5]:
X_data = X.drop('label',axis = 1)
X_data.head()

,f1,f2
0,0.450564,1.074305
1,0.085632,0.967682
2,0.117326,0.971521
3,0.982179,-0.380408
4,-0.720352,0.955850


In [6]:
print("Shape of X:",X_data.shape)
print("Shape of Y:",Y.shape)

Shape of X: (20000, 2)
Shape of Y: (20000,)


### Split data into train and test

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X_data,Y,test_size=0.30,stratify=Y)
Y_train = tf.keras.utils.to_categorical(Y_train, 2) 
Y_test = tf.keras.utils.to_categorical(Y_test, 2)
print("X_train Shape :",X_train.shape)
print("X_test Shape :",X_test.shape)
print("Y_train Shape :",Y_train.shape)
print("Y_test Shape :",Y_test.shape)

X_train Shape : (14000, 2)
X_test Shape : (6000, 2)
Y_train Shape : (14000, 2)
Y_test Shape : (6000, 2)


### Import all TensorFlow Libraries Used

In [8]:
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import TerminateOnNaN
import tensorflow.keras.backend as K
import datetime

### Function to store the Loss, Auc and Micro-F1 score in variable

In [9]:
class loss_metrics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.history={'Micro_F1': [],'AUC': [],'val_Micro_F1': [],'val_AUC': []}
    def on_epoch_end(self, epoch, logs={}):
        ## on end of each epoch, we will get logs and update the self.history dict    
        self.history['Micro_F1'].append(logs.get('Micro_F1'))
        self.history['AUC'].append(logs.get('AUC'))
        if logs.get('val_Micro_F1', -1) != -1:
            self.history['val_Micro_F1'].append(logs.get('val_Micro_F1'))
        if logs.get('val_AUC', -1) != -1:
            self.history['val_AUC'].append(logs.get('val_AUC'))


### Function to calculate micro-f1 score

In [10]:

def Micro_F1(y_true, y_pred):
    true_pos_class_0 = K.sum(K.round(y_pred[:,0]) * y_true[:,0])
    true_pos_class_1 = K.sum(K.round(y_pred[:,1]) * y_true[:,1])
    false_pos_class_0 = K.sum(K.round(y_pred[:,0])) - true_pos_class_0
    false_pos_class_1 = K.sum(K.round(y_pred[:,1])) - true_pos_class_1
    false_neg_class_0 = K.sum(y_true[:,0]) - true_pos_class_0
    false_neg_class_1 = K.sum(y_true[:,1]) - true_pos_class_1
    sum_true_pos = true_pos_class_0 + true_pos_class_1
    sum_false_pos = false_pos_class_0 + false_pos_class_1
    sum_false_neg = false_neg_class_0 + false_neg_class_1
    micro_precision = (sum_true_pos/(sum_true_pos + sum_false_pos))
    micro_recall = (sum_true_pos/(sum_true_pos + sum_false_neg))
    micro_F1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    #tf.print(K.round(y_pred[:,1]) & y_true[:,1])
    return micro_F1

### Function to adjust the learning rate on the conditions mentioned above

In [11]:
def changeLearningRate(epoch,lr):
    if(epoch > 1):
        if(history_own.history['val_Micro_F1'][-1] < history_own.history['val_Micro_F1'][-2]):
            lr = lr - ((10/100)*lr)
    if(epoch%3 == 0):
        lr = lr - ((5/100)*lr)
    return lr

### Model 1

In [12]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer1
layer1 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(input_layer)
#Dense hidden layer2
layer2 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer1)
#Dense hidden layer3
layer3 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer2)
#Dense hidden layer4
layer4 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer3)
#Dense hidden layer5
layer5 = Dense(50,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer4)
#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

#defining optimised
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,momentum = 0.5)

#Callbacks
#store values of log
history_own=loss_metrics() 
auc = tf.keras.metrics.AUC(name = 'AUC' )

#ModelCheckpoint = Saves the model when the acc. metric improves
filepath="model_save/weights-{epoch:02d}-{val_AUC:.4f}-{val_Micro_F1:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_Micro_F1',  verbose=1, save_best_only=True, mode='max')

#Lowers the learning rate systematically
lrschedule = LearningRateScheduler(changeLearningRate, verbose=10)

#Stops when the acc. metric does not imporve for 2 iterations
earlystop = EarlyStopping(monitor='val_Micro_F1', patience=2, verbose=15,mode='max')

#Creates tensorboard logs 
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

#terminates when the loss becomes NaN
TerminateWhenLossNaN = TerminateOnNaN()


model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=[auc,Micro_F1])

model.fit(X_train,Y_train,epochs=100, validation_data=(X_test,Y_test), batch_size=16,callbacks=[history_own,checkpoint,lrschedule,earlystop,TerminateWhenLossNaN,tensorboard_callback])

Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.009499999787658453.
875/875 [==============================] - 3s 2ms/step - loss: 0.6979 - AUC: 0.5233 - Micro_F1: 0.5196 - val_loss: 0.6899 - val_AUC: 0.5525 - val_Micro_F1: 0.5377

Epoch 00001: val_Micro_F1 improved from -inf to 0.53767, saving model to model_save\weights-01-0.5525-0.5377.hdf5
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 0.009499999694526196.
875/875 [==============================] - 1s 1ms/step - loss: 0.6936 - AUC: 0.5433 - Micro_F1: 0.5247 - val_loss: 0.6856 - val_AUC: 0.5525 - val_Micro_F1: 0.5377

Epoch 00002: val_Micro_F1 did not improve from 0.53767
Epoch 3/100

Epoch 00003: LearningRateScheduler setting learning rate to 0.009499999694526196.
875/875 [==============================] - 1s 1ms/step - loss: 0.6923 - AUC: 0.5433 - Micro_F1: 0.5243 - val_loss: 0.6908 - val_AUC: 0.5188 - val_Micro_F1: 0.5000

Epoch 00003: val_Micro_F1 did not improve from 0.53767

### Model 2

In [13]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer1
layer1 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(input_layer)
#Dense hidden layer2
layer2 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer1)
#Dense hidden layer3
layer3 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer2)
#Dense hidden layer4
layer4 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer3)
#Dense hidden layer5
layer5 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer4)
#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.RandomUniform(0, 1))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

#defining optimised
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1,momentum = 0.5)

#Callbacks
#store values of log
history_own=loss_metrics() 
auc = tf.keras.metrics.AUC(name = 'AUC' )

#ModelCheckpoint = Saves the model when the acc. metric improve
filepath="model_save/weights-{epoch:02d}-{val_AUC:.4f}-{val_Micro_F1:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_Micro_F1',  verbose=1, save_best_only=True, mode='max')

#Lowers the learning rate systematically
lrschedule = LearningRateScheduler(changeLearningRate, verbose=10)

#Stops when the acc. metric does not imporve for 2 iterations
earlystop = EarlyStopping(monitor='val_Micro_F1', patience=2, verbose=15,mode='max')

#Creates tensorboard logs 
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

#terminates when the loss becomes NaN
TerminateWhenLossNaN = TerminateOnNaN()


model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=[auc,Micro_F1])

model.fit(X_train,Y_train,epochs=100, validation_data=(X_test,Y_test), batch_size=16,callbacks=[history_own,checkpoint,lrschedule,earlystop,TerminateWhenLossNaN,tensorboard_callback])

Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.09500000141561031.
875/875 [==============================] - 4s 3ms/step - loss: 141.3216 - AUC: 0.4972 - Micro_F1: 0.5000 - val_loss: 0.6952 - val_AUC: 0.5000 - val_Micro_F1: 0.5000

Epoch 00001: val_Micro_F1 improved from -inf to 0.50000, saving model to model_save\weights-01-0.5000-0.5000.hdf5
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 0.0949999988079071.
875/875 [==============================] - 1s 1ms/step - loss: 0.6944 - AUC: 0.5020 - Micro_F1: 0.4993 - val_loss: 0.6958 - val_AUC: 0.5000 - val_Micro_F1: 0.5000

Epoch 00002: val_Micro_F1 did not improve from 0.50000
Epoch 3/100

Epoch 00003: LearningRateScheduler setting learning rate to 0.0949999988079071.
875/875 [==============================] - 1s 1ms/step - loss: 0.6947 - AUC: 0.5000 - Micro_F1: 0.5034 - val_loss: 0.6932 - val_AUC: 0.5000 - val_Micro_F1: 0.5000

Epoch 00003: val_Micro_F1 did not improve from 0.50000
Ep

### Model 3

In [14]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer1
layer1 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(input_layer)
#Dense hidden layer2
layer2 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(layer1)
#Dense hidden layer3
layer3 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(layer2)
#Dense hidden layer4
layer4 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(layer3)
#Dense hidden layer5
layer5 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(layer4)
#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.HeUniform())(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

#defining optimised
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,momentum = 0.5)

#Callbacks
#store values of log
history_own=loss_metrics() 
auc = tf.keras.metrics.AUC(name = 'AUC' )

#ModelCheckpoint = Saves the model when the acc. metric improve
filepath="model_save/weights-{epoch:02d}-{val_AUC:.4f}-{val_Micro_F1:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_Micro_F1',  verbose=1, save_best_only=True, mode='max')

#Lowers the learning rate systematically
lrschedule = LearningRateScheduler(changeLearningRate, verbose=10)

#Stops when the acc. metric does not imporve for 2 iterations
earlystop = EarlyStopping(monitor='val_Micro_F1', patience=2, verbose=15,mode='max')

#Creates tensorboard logs 
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

#terminates when the loss becomes NaN
TerminateWhenLossNaN = TerminateOnNaN()


model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=[auc,Micro_F1])

model.fit(X_train,Y_train,epochs=100, validation_data=(X_test,Y_test), batch_size=16,callbacks=[history_own,checkpoint,lrschedule,earlystop,TerminateWhenLossNaN,tensorboard_callback])

Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.009499999787658453.
875/875 [==============================] - 3s 3ms/step - loss: 0.6457 - AUC: 0.6781 - Micro_F1: 0.6246 - val_loss: 0.6230 - val_AUC: 0.7090 - val_Micro_F1: 0.6503

Epoch 00001: val_Micro_F1 improved from -inf to 0.65033, saving model to model_save\weights-01-0.7090-0.6503.hdf5
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 0.009499999694526196.
875/875 [==============================] - 1s 1ms/step - loss: 0.6124 - AUC: 0.7231 - Micro_F1: 0.6644 - val_loss: 0.6087 - val_AUC: 0.7297 - val_Micro_F1: 0.6645

Epoch 00002: val_Micro_F1 improved from 0.65033 to 0.66450, saving model to model_save\weights-02-0.7297-0.6645.hdf5
Epoch 3/100

Epoch 00003: LearningRateScheduler setting learning rate to 0.009499999694526196.
875/875 [==============================] - 1s 1ms/step - loss: 0.6082 - AUC: 0.7287 - Micro_F1: 0.6692 - val_loss: 0.6048 - val_AUC: 0.7327 - val_Micro_F1: 

### Model 4

In [15]:
#Input layer
input_layer = Input(shape=(2,))
#Dense hidden layer1
layer1 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(input_layer)
#Dense hidden layer2
layer2 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(layer1)
#Dense hidden layer3
layer3 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(layer2)
#Dense hidden layer4
layer4 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(layer3)
#Dense hidden layer5
layer5 = Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.HeUniform())(layer4)
#output layer
output = Dense(2,activation='softmax',kernel_initializer=tf.keras.initializers.HeUniform())(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

#defining optimised
optimizer = tf.keras.optimizers.Adam()

#Callbacks
#store values of log
history_own=loss_metrics() 
auc = tf.keras.metrics.AUC(name = 'AUC' )

#ModelCheckpoint = Saves the model when the acc. metric improve
filepath="model_save/weights-{epoch:02d}-{val_AUC:.4f}-{val_Micro_F1:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_Micro_F1',  verbose=1, save_best_only=True, mode='max')

#Lowers the learning rate systematically
lrschedule = LearningRateScheduler(changeLearningRate, verbose=10)

#Stops when the acc. metric does not imporve for 2 iteration
earlystop = EarlyStopping(monitor='val_Micro_F1', patience=2, verbose=15,mode='max')

#Creates tensorboard logs 
log_dir="logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

#terminates when the loss becomes NaN
TerminateWhenLossNaN = TerminateOnNaN()


model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=[auc,Micro_F1])

model.fit(X_train,Y_train,epochs=100, validation_data=(X_test,Y_test), batch_size=10,callbacks=[history_own,checkpoint,lrschedule,earlystop,TerminateWhenLossNaN,tensorboard_callback])

Epoch 1/100

Epoch 00001: LearningRateScheduler setting learning rate to 0.0009500000451225787.
1400/1400 [==============================] - 5s 3ms/step - loss: 0.6472 - AUC: 0.6678 - Micro_F1: 0.6159 - val_loss: 0.6173 - val_AUC: 0.7159 - val_Micro_F1: 0.6488

Epoch 00001: val_Micro_F1 improved from -inf to 0.64883, saving model to model_save\weights-01-0.7159-0.6488.hdf5
Epoch 2/100

Epoch 00002: LearningRateScheduler setting learning rate to 0.0009500000160187483.
1400/1400 [==============================] - 2s 1ms/step - loss: 0.6120 - AUC: 0.7232 - Micro_F1: 0.6614 - val_loss: 0.6019 - val_AUC: 0.7371 - val_Micro_F1: 0.6697

Epoch 00002: val_Micro_F1 improved from 0.64883 to 0.66967, saving model to model_save\weights-02-0.7371-0.6697.hdf5
Epoch 3/100

Epoch 00003: LearningRateScheduler setting learning rate to 0.0009500000160187483.
1400/1400 [==============================] - 2s 1ms/step - loss: 0.6099 - AUC: 0.7265 - Micro_F1: 0.6624 - val_loss: 0.6027 - val_AUC: 0.7370 - val_M

In [16]:
%reload_ext tensorboard

In [18]:
%tensorboard --logdir logs/fit/

Reusing TensorBoard on port 6006 (pid 7920), started 0:01:29 ago. (Use '!kill 7920' to kill it.)